# QUBO Portfolio Optimisation for a Single Football Match

The odds and implied probability of twenty different markets related to a single football match are pulled via API. We have taken an hourly snapshot of the odds for these markets, and the correlation analysis of these for 3 days up to the time of the match was carried out. 

With odds and implied probability, an expectation value of the bet put is then calculated and assigned to the nodes of the graph we are building as w_i.

In [1]:
import numpy as np
import pandas as pd
import itertools
import time
import orbit 

ModuleNotFoundError: No module named 'orbit'

In [2]:
def expected_value(p, odds):
    """
    Expected profit per unit stake:
    EV = p * odds - 1
    """
    return p * odds - 1.0

def portfolio_energy_qubo(x, w, Q):
    """
    QUBO energy:
    E(x) = sum_i w_i x_i + sum_{i<j} Q_ij x_i x_j
    x : 1D numpy array of 0/1 of length N
    """
    linear = np.dot(w, x)
    quadratic = 0.0
    N = len(x)
    for i in range(N):
        for j in range(i+1, N):
            quadratic += Q[i, j] * x[i] * x[j]
    return linear + quadratic

In [3]:
#TO BE REPLACED BY THE API DATA
def generate_betting_timeseries(
    match_time="2025-01-10 20:00",
    n_days_before=3,
    n_markets=21,
    seed=42
):
    """
    Create a time series DataFrame of hourly odds and implied probabilities
    for multiple betting markets in the days leading up to a match.
    
    Parameters
    ----------
    match_time : str or pd.Timestamp
        Kick-off time of the match.
    n_days_before : int
        Number of days before the match to include (hourly intervals).
    n_markets : int
        Number of distinct betting markets.
    seed : int
        Random seed for reproducibility.
    
    Returns
    -------
    df : pd.DataFrame
        Long-format DataFrame with columns:
        ['timestamp', 'market', 'odds', 'implied_prob'].
    """
    np.random.seed(seed)
    
    # Time index: hourly from (match_time - n_days_before) to match_time
    match_time = pd.to_datetime(match_time)
    start_time = match_time - pd.Timedelta(days=n_days_before)
    time_index = pd.date_range(start=start_time, end=match_time, freq="H")
    
    # Create some market names (you can replace with real ones)
    markets = [f"Market_{i+1:02d}" for i in range(n_markets)]
    
    # DataFrame to hold odds paths (wide format first)
    odds_wide = pd.DataFrame(index=time_index, columns=markets, dtype=float)
    
    for m in markets:
        # Random initial odds between 1.5 and 6.0
        start_odds = np.random.uniform(1.5, 6.0)
        
        # Random walk on log-odds to keep them positive and fairly smooth
        n_steps = len(time_index)
        noise = np.random.normal(loc=0.0, scale=0.03, size=n_steps)  # step volatility
        log_odds = np.log(start_odds) + np.cumsum(noise)
        series_odds = np.exp(log_odds)
        
        # Clip odds to a sensible range
        series_odds = np.clip(series_odds, 1.01, 20.0)
        
        odds_wide[m] = series_odds
    
    # Long / tidy format
    df = (
        odds_wide
        .reset_index()
        .melt(id_vars="index", var_name="market", value_name="odds")
        .rename(columns={"index": "timestamp"})
    )
    
    # Implied probability from decimal odds: p = 1 / odds
    df["implied_prob"] = 1.0 / df["odds"]
    
    return df


if __name__ == "__main__":
    df = generate_betting_timeseries()
    print(df.head())
    print("\nShape:", df.shape)


            timestamp     market      odds  implied_prob
0 2025-01-07 20:00:00  Market_01  3.080929      0.324577
1 2025-01-07 21:00:00  Market_01  3.110546      0.321487
2 2025-01-07 22:00:00  Market_01  3.136694      0.318807
3 2025-01-07 23:00:00  Market_01  3.233240      0.309287
4 2025-01-08 00:00:00  Market_01  3.177385      0.314724

Shape: (1533, 4)


/var/folders/18/wyzslnfs5d12r_xpx6n0pq900000gn/T/ipykernel_72343/822222000.py:34: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_index = pd.date_range(start=start_time, end=match_time, freq="H")


In [4]:
def compute_market_correlation(df, value_col="implied_prob"):
    """
    Compute the correlation matrix of N betting markets based on
    implied probabilities (or odds).

    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns ['timestamp', 'market', value_col]
    value_col : str
        Column name to compute correlation on ("implied_prob" or "odds")

    Returns
    -------
    corr_matrix : pd.DataFrame
        N×N correlation matrix of markets.
    wide_df : pd.DataFrame
        Time-indexed wide DataFrame used for correlation.
    """

    wide_df = df.pivot_table(
        index="timestamp",
        columns="market",
        values=value_col
    )
    corr_matrix = wide_df.corr()
    return corr_matrix, wide_df

if __name__ == "__main__":
    corr, wide = compute_market_correlation(df)
    print("\n=== Correlation Matrix (20×20) ===")
    print(corr)


=== Correlation Matrix (20×20) ===
market     Market_01  Market_02  Market_03  Market_04  Market_05  Market_06  \
market                                                                        
Market_01   1.000000  -0.020199  -0.082113  -0.123655   0.151383  -0.259624   
Market_02  -0.020199   1.000000  -0.318134  -0.085121  -0.347548   0.410202   
Market_03  -0.082113  -0.318134   1.000000  -0.639656   0.844664   0.314821   
Market_04  -0.123655  -0.085121  -0.639656   1.000000  -0.633921  -0.641922   
Market_05   0.151383  -0.347548   0.844664  -0.633921   1.000000   0.182712   
Market_06  -0.259624   0.410202   0.314821  -0.641922   0.182712   1.000000   
Market_07   0.125228   0.547869  -0.722832   0.179351  -0.636206   0.041396   
Market_08   0.649443   0.197102  -0.331304   0.129960  -0.192612  -0.298146   
Market_09  -0.349277   0.097443  -0.173095   0.091511  -0.237146   0.379116   
Market_10  -0.170570   0.157804  -0.718524   0.455711  -0.650256  -0.234231   
Market_11   0.30

## Implementation of the correlation calculations on real API data

In [3]:
# Convert CSV outputs to DataFrames, create an implied probability and expected value dataframe
# Then run correlation calculations on the implied probability dataframe

# Import the CSV data and convert to pandas DataFrame
import pandas as pd

bar_v_atl_df = pd.read_csv('output-bar-v-atl.csv')

# Convert timestamp column to datetime
bar_v_atl_df['timestamp'] = pd.to_datetime(bar_v_atl_df['timestamp'])

# Display basic information about the dataframe
print("DataFrame shape:", bar_v_atl_df.shape)
print("\nFirst few rows:")
print(bar_v_atl_df.head())
print("\nColumn names:")
print(bar_v_atl_df.columns.tolist())

DataFrame shape: (168, 42)

First few rows:
            timestamp  money_line_home  money_line_draw  money_line_away  \
0 2025-11-23 16:34:21            1.740             4.13             4.02   
1 2025-11-23 16:35:21            1.746             4.13             4.01   
2 2025-11-23 16:38:01            1.769             4.07             3.95   
3 2025-11-23 16:38:31            1.793             4.01             3.89   
4 2025-11-23 16:44:46            1.781             4.02             3.94   

   spread_-1.75_home  spread_-1.75_away  spread_-1.5_home  spread_-1.5_away  \
0               3.47              1.297              2.86             1.409   
1               3.49              1.295              2.87             1.408   
2               3.55              1.288              2.92             1.396   
3               3.61              1.280              2.97             1.384   
4               3.58              1.283              2.94             1.390   

   spread_-1.25_home  sp

In [6]:
# Convert the wide-format dataframe to long format
# Transform from: timestamp, money_line_home, money_line_draw, ... (wide)
# To: timestamp, market, odds, implied_prob, expected_value (long)

# Melt the dataframe: keep timestamp as id, convert all odds columns to rows
df_long = bar_v_atl_df.melt(
    id_vars=['timestamp'],
    var_name='market',
    value_name='odds'
)

# Calculate implied probability: p = 1 / odds
# This represents the probability implied by the decimal odds
df_long['implied_prob'] = 1.0 / df_long['odds']

# Calculate expected value: EV = p * odds - 1
# This represents the expected profit per unit stake
# EV > 0 means positive expected value (profitable bet)
# EV < 0 means negative expected value (unprofitable bet)
df_long['expected_value'] = df_long['implied_prob'] * df_long['odds'] - 1.0

# Display the transformed dataframe
print("Transformed DataFrame (long format):")
print(df_long.head(10))
print(f"\nDataFrame shape: {df_long.shape}")
print(f"\nNumber of unique markets: {df_long['market'].nunique()}")
print(f"\nNumber of unique timestamps: {df_long['timestamp'].nunique()}")

# Show a sample of calculations to verify
print("\n=== Sample Calculations Verification ===")
sample = df_long.head(5)
for idx, row in sample.iterrows():
    odds = row['odds']
    implied_prob = row['implied_prob']
    expected_value = row['expected_value']
    print(f"\nMarket: {row['market']}")
    print(f"  Odds: {odds:.4f}")
    print(f"  Implied Probability = 1 / {odds:.4f} = {implied_prob:.6f}")
    print(f"  Expected Value = {implied_prob:.6f} * {odds:.4f} - 1 = {expected_value:.6f}")
    print(f"  Verification: {implied_prob * odds - 1.0:.6f} (should match expected_value)")

# Store the long format dataframe for later use
df_bar_v_atl_probs = df_long.copy()

Transformed DataFrame (long format):
            timestamp           market   odds  implied_prob  expected_value
0 2025-11-23 16:34:21  money_line_home  1.740      0.574713   -1.110223e-16
1 2025-11-23 16:35:21  money_line_home  1.746      0.572738   -1.110223e-16
2 2025-11-23 16:38:01  money_line_home  1.769      0.565291    0.000000e+00
3 2025-11-23 16:38:31  money_line_home  1.793      0.557724    0.000000e+00
4 2025-11-23 16:44:46  money_line_home  1.781      0.561482    0.000000e+00
5 2025-11-23 16:44:56  money_line_home  1.763      0.567215    0.000000e+00
6 2025-11-24 04:08:14  money_line_home  1.787      0.559597   -1.110223e-16
7 2025-11-24 04:08:24  money_line_home  1.793      0.557724    0.000000e+00
8 2025-11-24 16:13:10  money_line_home  1.781      0.561482    0.000000e+00
9 2025-11-29 21:35:28  money_line_home  1.781      0.561482    0.000000e+00

DataFrame shape: (6888, 5)

Number of unique markets: 41

Number of unique timestamps: 168

=== Sample Calculations Verificati

In [ ]:
# Run correlation calculations on the implied probability dataframe
corr_bar_v_atl_df = df_bar_v_atl_probs.pivot_table(
    index='timestamp',
    columns='market',
    values='implied_prob'
).corr()
print("\nCorrelation Matrix:")
print(corr_bar_v_atl_df)


Correlation Matrix:
market             money_line_away  money_line_draw  money_line_home  \
market                                                                 
money_line_away           1.000000         0.679587        -0.900308   
money_line_draw           0.679587         1.000000        -0.656940   
money_line_home          -0.900308        -0.656940         1.000000   
spread_-0.25_away         0.978194         0.789322        -0.932326   
spread_-0.25_home        -0.913947        -0.604573         0.985406   
spread_-0.5_away          0.972703         0.808371        -0.923486   
spread_-0.5_home         -0.896765        -0.678509         0.992860   
spread_-0.75_away         0.972731         0.814077        -0.915507   
spread_-0.75_home        -0.907498        -0.681312         0.991780   
spread_-1.0_away          0.966216         0.815460        -0.905421   
spread_-1.0_home         -0.926131        -0.722805         0.983098   
spread_-1.25_away         0.949878         

In [ ]:
# All relevant dataframes are now created, here they are:
# bar_v_atl_df ==> the original dataframe with the odds
# df_long ==> the long format dataframe with the implied probabilities and expected values
# df_bar_v_atl_probs ==> the long format dataframe with the implied probabilities
# corr_bar_v_atl_df ==> the correlation matrix of the implied probabilities

In [ ]:
#Taking the lastest odds from the API
final_snapshot = (
    df.sort_values("timestamp")
      .groupby("market")
      .tail(1)
      .set_index("market")
)
final_snapshot

In [ ]:
p_final = final_snapshot["implied_prob"].values
odds_final = final_snapshot["odds"].values

# Expected value per market
EV = p_final * odds_final - 1.0 #type: ignore

# Linear QUBO term
w = -EV

In [ ]:
wide = df.pivot_table(
    index="timestamp",
    columns="market",
    values="implied_prob"
).sort_index()

rho = wide.corr().values  # 20 × 20 matrix
Q = rho

In [ ]:
#Brute-Force Calculations
N = len(final_snapshot)
all_results = []

start_time = time.perf_counter()  # start timer

for bits in itertools.product([0, 1], repeat=N):
    x = np.array(bits)
    E = -1 * portfolio_energy_qubo(x, w, Q)
    all_results.append({
        "x": bits,
        "E": E,
        "num_bets": x.sum()
    })

end_time = time.perf_counter()  # end timer
elapsed_bf = end_time - start_time

results_df = (
    pd.DataFrame(all_results)
      .sort_values("E", ascending=True)
      .reset_index(drop=True)
)

print(results_df.head(32))
print(f"\nChecked {2**N} configurations for N={N}.")
print(f"Time to find ground state (brute force): {elapsed_bf:.4f} seconds")

In [ ]:
#Orbit Calculations
N = len(final_snapshot)
bets = final_snapshot.index.tolist()  

# Standard QUBO -> Ising mapping 
J0 = Q / 4.0
h0 = np.zeros(N)
for i in range(N):
    h0[i] = w[i] / 2.0 + 0.25 * np.sum(Q[i, :])

ising_J = J0
ising_h = h0

start_time = time.perf_counter()
result = orbit.optimize_ising(
    -1*ising_J,
    -1*ising_h,
    n_replicas=3,
    full_sweeps=50_000,
    beta_initial=0.35,
    beta_end=3.5,
    beta_step_interval=1,
)

elapsed_orb = time.perf_counter() - start_time

s_star = np.array(result.min_state)  
x_star = (1 + s_star) // 2                 

E_orbit = 1 * portfolio_energy_qubo(x_star, w, Q)

chosen_bets = [b for b, bit in zip(bets, x_star) if bit == 1]

print("=== ORBIT optimisation result ===")
print(f"Time to (approximately) find ground state: {elapsed_orb:.4f} seconds\n")

print("Spins (s*):", s_star.tolist())
print("Bits  (x*):", x_star.tolist())
print("Number of bets in portfolio:", int(x_star.sum()))
print("Selected bets:", chosen_bets)

print("\nORBIT reported min_cost:", result.min_cost)
print("Objective E = -portfolio_energy_qubo(x*, w, Q):", E_orbit)

# Optional: compare with brute-force ground state if you still have results_df
if 'results_df' in globals():
    print("\n=== Comparison with brute force ===")
    print("Brute-force best E:", results_df.loc[0, 'E'])
    print("Brute-force best x:", results_df.loc[0, 'x'])

In [ ]:
# --- Check ORBIT solution against brute-force results and rank it ---


orbit_bits = tuple(int(b) for b in x_star)
matches = results_df[results_df["x"] == orbit_bits]

if matches.empty:
    print("⚠ ORBIT bitstring not found in brute-force results (this should not happen if N matches).")
else:
    match_idx = matches.index[0]   # 0-based index in results_df
    rank = match_idx + 1           # human-friendly rank (1 = best)

    E_best = results_df.loc[0, "E"]
    x_best = results_df.loc[0, "x"]
    E_orbit_bruteforce = matches.iloc[0]["E"]

    print("\n=== ORBIT vs Brute Force ===")
    print(f"ORBIT bitstring: {orbit_bits}")
    print(f"ORBIT energy from brute-force table: {E_orbit_bruteforce:.6f}")
    print(f"Ground state energy (brute force):   {E_best:.6f}")
    print(f"Ground state bitstring:              {x_best}")

    print(f"\nRank of ORBIT state among all 2^{N} configs: {rank} (1 = ground state)")
    print(f"Energy gap to ground state: {E_orbit_bruteforce - E_best:.6f}")

    # Optional: how many configs are strictly better / equal
    n_better = (results_df["E"] < E_orbit_bruteforce).sum()
    n_equal  = (results_df["E"] == E_orbit_bruteforce).sum()
    print("")
    print(f"Time to the lowest state using orbit: {elapsed_orb:.4f} seconds\n")
    print(f"Time to find the ground state using brute-force  : {elapsed_bf:.4f} seconds\n")